In [1]:
import numpy as np
import trimesh
import tqdm 
import open3d as o3d

In [2]:
import math
import random

def cal_area(p1, p2, p3): 
    V_AB = p2 - p1
    V_AC = p3 - p1

    cross_product = np.cross(V_AB, V_AC)
    #area = 0.5 * np.linalg.norm(cross_product, axis=1)
    area = 0.5 * np.linalg.norm(cross_product,axis=1)

    return area

def uniform_sampling_from_mesh(vertices, faces, sample_num):
    # -------- TODO -----------
    # 1. compute area of each triangles
    # 2. compute probability of each triangles from areas
    # 3. sample N faces according to the probability
    # 4. for each face, sample 1 point
    # Note that FOR-LOOP is not allowed!
    # -------- TODO -----------
    p1 = vertices[faces[:,0]] # 每个face的第一个点 n*点的3个坐标
    p2 = vertices[faces[:,1]]
    p3 = vertices[faces[:,2]]
    area = cal_area(p1,p2,p3)

    prob = area / np.sum(area)

    # sample N faces according to the probability
    samples = random.choices(range(len(prob)), weights=prob, k=sample_num) # 512
    # 选择每个被选中face的三角形重心作为采样点
    uniform_pc = np.mean(vertices[faces[samples]], axis = 1)

    return area, prob, uniform_pc
        

In [8]:
def farthest_point(p_set,points):
    # 到S集合最近的点的距离最大
    dis = np.min(np.sum(np.square(p_set-points[:,np.newaxis,:]),axis = 2),axis =1)
    return np.argmax(dis)


def farthest_point_sampling(pc, sample_num):
    # -------- TODO -----------
    # FOR LOOP is allowed here.
    # -------- TODO -----------
    U = pc
    
    U_size = U.shape[0]
    # 随便选一个初始的点
    idx = np.random.randint(0, U_size)
    p = U[idx]
    S = p

    for i in range(sample_num-1):
        idx = farthest_point(S,U)
        p = U[idx]
        S = np.vstack((S, p))
        
    S = np.array(S)
    results = S
    
    return results

In [5]:
# task 1: uniform sampling 

obj_path = 'spot.obj'
mesh = trimesh.load(obj_path)
print('faces shape: ', mesh.faces.shape)
sample_num = 512
area, prob, uniform_pc = uniform_sampling_from_mesh(mesh.vertices, mesh.faces, sample_num)

# Visualization. For you to check your code
#pcd = o3d.geometry.PointCloud()
#pcd.points = o3d.utility.Vector3dVector(uniform_pc)
#o3d.visualization.draw_geometries([pcd])

np.savetxt('uniform_sampling_vis.txt', uniform_pc)

print('area shape: ',area.shape)
print('prob shape: ',prob.shape)
print('pc shape: ',uniform_pc.shape)
# the result should satisfy: 
#       area.shape = (13712, ) 
#       prob.shape = (13712, ) 
#       uniform_pc.shape = (512, 3) 

# For submission
save_dict = {'area': area, 'prob': prob, 'pc': uniform_pc}
np.save('../results/uniform_sampling_results', save_dict)

faces shape:  (13712, 3)
area shape:  (13712,)
prob shape:  (13712,)
pc shape:  (512, 3)


In [9]:
# task 2: FPS

init_sample_num = 2000
final_sample_num = 512
_,_, tmp_pc = uniform_sampling_from_mesh(mesh.vertices, mesh.faces, init_sample_num)
fps_pc = farthest_point_sampling(tmp_pc, final_sample_num)

# Visualization. For you to check your code
np.savetxt('fps_vis.txt', fps_pc)
#pcd = o3d.geometry.PointCloud()
#pcd.points = o3d.utility.Vector3dVector(fps_pc)
#o3d.visualization.draw_geometries([pcd])

# For submission
np.save('../results/fps_results', fps_pc)

In [29]:
# task 3: metrics

from earthmover import earthmover_distance   # EMD may be very slow (1~2mins)
from scipy.stats import wasserstein_distance
from scipy.spatial.distance import cdist
from scipy.optimize import linear_sum_assignment
# -----------TODO---------------
# compute chamfer distance and EMD for two point clouds sampled by uniform sampling and FPS.
# sample and compute CD and EMD again. repeat for five times.
# save the mean and var.
# -----------TODO---------------

def chamfer_dis(s1,s2): #(512,3)
    dis = s1[np.newaxis,:,:] - s2[:,np.newaxis,:]
    dis = np.sqrt(np.sum(np.square(dis),axis=2))
    d1 = np.mean(np.min(dis,axis = 0))
    d2 = np.mean(np.min(dis,axis = 1))
    return (d1 + d2) / 2

CD = chamfer_dis(uniform_pc,fps_pc)
print(CD)

CDs = []
EMDs = []
for i in range(5):
    area, prob, uniform_pc = uniform_sampling_from_mesh(mesh.vertices, mesh.faces, sample_num)
    fps_pc = farthest_point_sampling(tmp_pc, final_sample_num)

    CD = chamfer_dis(uniform_pc,fps_pc)
    EMD = earthmover_distance(list(map(tuple, uniform_pc)), list(map(tuple, fps_pc)))
    
    CDs.append(CD)
    EMDs.append(EMD)
    print(CD)
    print(EMD)
    print('')

CD_mean = np.mean(CDs) 
CD_var = np.var(CDs)
print("CD_mean:",CD_mean)
print("CD_var:",CD_var)

EMD_mean = np.mean(EMDs) 
EMD_var = np.var(EMDs)
print("EMD_mean:",EMD_mean)
print("EMD_var:",EMD_var)

CD_mean = 0
CD_var = 0
EMD_mean = 0
EMD_var = 0

# For submission
np.save('../results/metrics', {'CD_mean':CD_mean, 'CD_var':CD_var, 'EMD_mean':EMD_mean, 'EMD_var':EMD_var})

1.3909749899338069
move 0.001953125 dirt from (49.153261289999996, 31.443989216666665, 36.69913893) to (48.40746714666667, 33.16622971666666, 34.333333333333336) for a cost of 0.005898100338721639
move 0.001953125 dirt from (26.433700916666666, 15.861930546666665, 30.333333333333332) to (26.370866396666667, 15.829392476666667, 31.333333333333332) for a cost of 0.0019580084485104126
move 0.001953125 dirt from (17.27774289, 47.333333333333336, 19.392060086666667) to (16.48107153333333, 47.333333333333336, 20.333333333333332) for a cost of 0.0024085132411836817
move 0.001953125 dirt from (15.333333333333334, 34.666666666666664, 22.698259273333335) to (14.207269266666666, 35.511883553333334, 22.055950006666666) for a cost of 0.0030225977552523894
move 0.001953125 dirt from (34.333333333333336, 24.666666666666668, 42.62647887333333) to (34.666666666666664, 26.333333333333332, 42.29661908666667) for a cost of 0.0033816126533393496
move 0.001953125 dirt from (25.666666666666668, 13.3333333333